# Plant Disease Classification with Fine-Tuned YOLOv8

Notebook by Aditya Tapshalkar

Kaggle dataset: https://www.kaggle.com/datasets/vipoooool/new-plant-diseases-dataset

---

## Abstract

## Introduction


In [9]:
%env KAGGLEHUB_CACHE=/mnt/devbox/kaggle

import os

import kagglehub
import pandas as pd
import torch

torch.manual_seed(42)
print("Torch version:", torch.__version__)
print("Current device:", "cuda" if torch.cuda.is_available() else "cpu")

env: KAGGLEHUB_CACHE=/mnt/devbox/kaggle
Torch version: 2.5.1
Current device: cuda


In [10]:
# Downloading dataset
path = kagglehub.dataset_download("vipoooool/new-plant-diseases-dataset")
print("Path to dataset files:", path)

Download already complete (2897709187 bytes).
Extracting files...
Path to dataset files: /mnt/devbox/kaggle/datasets/vipoooool/new-plant-diseases-dataset/versions/2
